In [ ]:
# from sklearn.datasets import fetch_openml

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
mnist = fetch_openml('mnist_784', version = 1)

In [ ]:
mnist.keys()

In [ ]:
X, y = mnist['data'], mnist['target']


In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt


In [ ]:
some_digit = X.iloc[0].values

some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image)
plt.axis("off")
plt.show()

In [ ]:
y.iloc[0]

In [ ]:
y = y.astype(np.uint8)

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)


In [ ]:
from sklearn.linear_model import SGDClassifier 

In [ ]:
sgd_clf = SGDClassifier(random_state= 42)
sgd_clf.fit(X_train, y_train_5)

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
from sklearn.model_selection import cross_val_score 
cross_val_score(sgd_clf, X_train, y_train_5, cv = 3, scoring = 'accuracy')

In [ ]:
from sklearn.base import BaseEstimator 

class Never5Classifier(BaseEstimator):
    def fit(self, X, y = None):
        pass
    
    def predict(self, X):
        return np.zeros((len(X), 1), dtype = bool)
    
    

In [ ]:
never_5_clf= Never5Classifier()

cross_val_score(never_5_clf, X_train, y_train_5, cv = 3, scoring= 'accuracy')

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv = 3)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision_score(y_train_5, y_train_pred)

In [ ]:
recall_score(y_train_5, y_train_pred)

In [ ]:
from sklearn.metrics import f1_score 
f1_score(y_train_5, y_train_pred)

In [ ]:
y_score = cross_val_predict(sgd_clf, X_train, y_train_5, cv = 3, method = 'decision_function')

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_score)

In [ ]:
def precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label = "Precision")
    plt.plot(thresholds, recalls[:-1], 'g-', label = 'Recall')
    plt.axis()
    plt.legend
    
precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)] # ~7816

In [ ]:
print(threshold_90_precision )

In [ ]:
y_train_pred_90 = (y_score >= threshold_90_precision)

In [ ]:
precision_score(y_train_5, y_train_pred_90)

In [ ]:
recall_score(y_train_5, y_train_pred_90)

In [ ]:
from sklearn.metrics import roc_curve 

fpr, tpr, thresholds = roc_curve(y_train_5, y_score)

In [ ]:
def plot_roc_curve(fpr, tpr, label = None):
    plt.plot(fpr, tpr, linewidth = 2, label = label)
    plt.plot([0,1], [0,1], 'k--')
    
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train_5, y_score)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state  = 42)
y_prob = cross_val_predict(forest_clf, X_train, y_train_5, cv = 3, method = 'predict_proba')

In [ ]:
y_scores_forest= y_prob[:,1]
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_5, y_scores_forest)

In [ ]:
plt.plot(fpr, tpr, "b:", label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right")
plt.show()

In [ ]:
roc_auc_score(y_train_5, y_scores_forest)

In [ ]:
sgd_clf.fit(X_train, y_train)



In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
some_digit_score = sgd_clf.decision_function([some_digit])

In [ ]:
some_digit_score